In [1]:
import csv
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
from nltk.corpus import wordnet

In [5]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.2 MB/s eta 0:00:00


In [8]:

import csv
import pandas as pd
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
# from keras.layers.embeddings import Embedding

## Plot
# import plotly.offline as py
# import plotly.graph_objs as go
# py.init_notebook_mode(connected=True)
# import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk import word_tokenize



# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn import model_selection, preprocessing, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn.svm import SVC
from keras.models import load_model
import pickle
# Sampling
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

import contractions
import re, string, unicodedata
import nltk
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping

# #wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# The code starts by importing necessary libraries, setting up Google Drive, and downloading required NLTK resources.
# It then imports additional libraries for text processing, machine learning, and handling imbalanced datasets.
# The read_bias_data function is defined to read bias data from two CSV files, drop unnecessary columns, and concatenate relevant text columns.

In [ ]:
# The label_encode function is defined to encode bias labels with numerical values for machine learning models.

In [9]:

def read_bias_data(f1, f2):
  cols=["title", "text", "sen_obj", "source", "bias"]
  df = pd.read_csv(f1, skiprows=1, dtype=str, names=cols)
  tdf = pd.read_csv(f2, skiprows=1, dtype=str, names=cols)
  df=df.drop(["sen_obj"], 1)
  tdf=tdf.drop(["sen_obj"], 1)
  df["all"]=df.source + " " + df.title + " "+ df.text
  tdf["all"]=tdf.source + " " + tdf.title + " "+ tdf.text
  df=df.dropna()
  tdf=tdf.dropna()
  return df, tdf

def label_encode(df, tdf):
  tdf["bias"]= tdf["bias"].replace("lc", 1)
  tdf["bias"]= tdf["bias"].replace("rc", 2)
  tdf["bias"]= tdf["bias"].replace("left", 1)
  tdf["bias"]= tdf["bias"].replace("right", 2)
  tdf["bias"]= tdf["bias"].replace("least-biased",0)
  df["bias"]= df["bias"].replace("1", 1)
  df["bias"]= df["bias"].replace("2", 2)
  df["bias"]= df["bias"].replace("0", 0)
  return df, tdf

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)  # Use get() with a default value



In [ ]:
# Two functions, get_wordnet_pos and clean_text, are defined for part-of-speech tagging and text cleaning.

In [10]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove digits
    text = re.sub(r'\d', '', text)

    # Remove stop words
    stops = set(stopwords.words("english"))
    text = ' '.join([word for word in word_tokenize(text) if word.lower() not in stops])

    # Use BeautifulSoup to handle HTML entities
    text = BeautifulSoup(text, "html.parser").get_text()

    # Fix contractions
    text = contractions.fix(text)

    # Handle specific contractions and substitutions
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    lemmed_words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in word_tokenize(text)]
    text = " ".join(lemmed_words)

    return text


In [11]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/output.csv')

In [12]:
df

,headline,contains_gpe,sentiment,contains_numbers,contains_personal_pronouns,label
0,Should I Get Bings,0,0.0000,0,0,1
1,Which TV Female Friend Group Do You Belong In,0,0.4939,0,1,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",0,-0.5574,0,1,1
3,"This Vine Of New York On ""Celebrity Big Brothe...",0,0.6115,0,0,1
4,A Couple Did A Stunning Photo Shoot With Their...,0,-0.3400,0,0,1
...,...,...,...,...,...,...
31995,"To Make Female Hearts Flutter in Iraq, Throw a...",1,0.0000,0,0,0
31996,"British Liberal Democrat Patsy Calton, 56, die...",0,-0.6597,1,0,0
31997,Drone smartphone app to help heart attack vict...,0,-0.4019,0,0,0
31998,"Netanyahu Urges Pope Benedict, in Israel, to D...",1,-0.3400,0,0,0


In [13]:
df.rename(columns={'headline': 'text'}, inplace=True)
df.rename(columns={'clickbait': 'label'}, inplace=True)

In [14]:
df["all_cleaned"]=df["text"].apply(clean_text)

In [15]:
df

,text,contains_gpe,sentiment,contains_numbers,contains_personal_pronouns,label,all_cleaned
0,Should I Get Bings,0,0.0000,0,0,1,get bings
1,Which TV Female Friend Group Do You Belong In,0,0.4939,0,1,1,tv female friend group belong
2,"The New ""Star Wars: The Force Awakens"" Trailer...",0,-0.5574,0,1,1,new star war force awakens trailer give chill
3,"This Vine Of New York On ""Celebrity Big Brothe...",0,0.6115,0,0,1,vine new york celebrity big brother fuck perfect
4,A Couple Did A Stunning Photo Shoot With Their...,0,-0.3400,0,0,1,couple stun photo shoot baby learn inoperable ...
...,...,...,...,...,...,...,...
31995,"To Make Female Hearts Flutter in Iraq, Throw a...",1,0.0000,0,0,0,make female heart flutter iraq throw shoe
31996,"British Liberal Democrat Patsy Calton, 56, die...",0,-0.6597,1,0,0,british liberal democrat patsy calton dy cancer
31997,Drone smartphone app to help heart attack vict...,0,-0.4019,0,0,0,drone smartphone app help heart attack victim ...
31998,"Netanyahu Urges Pope Benedict, in Israel, to D...",1,-0.3400,0,0,0,netanyahu urge pope benedict israel denounce iran


In [16]:
# Test data is read, text cleaning is applied, and the evaluate_clickbait_pip function is called to assess the model's performance on the test data.

In [17]:
tdf = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/scrapped_data.csv")

In [18]:
tdf

,headline,label
0,Democrat Sheila Jackson Lee defeated in conten...,0
1,Deadly tornado outbreak leaves extensive damag...,0
2,LSU QB beats out competitors to take home cove...,0
3,Princess Charlotte takes center stage in Princ...,0
4,Trump moves into clear lead as Biden's approva...,0
...,...,...
280,“It put me into a lot of danger and I hurt a l...,1
281,“Kris getting Tristan a deal with ESPN??? Afte...,1
282,“Beyoncé and Taylor just told y’all to shut up...,1
283,Court documents revealed that Joe and Sophie w...,1


In [19]:
tdf["all_cleaned"]=tdf["headline"].apply(clean_text)


In [20]:
tdf

,headline,label,all_cleaned
0,Democrat Sheila Jackson Lee defeated in conten...,0,democrat sheila jackson lee defeat contentious...
1,Deadly tornado outbreak leaves extensive damag...,0,deadly tornado outbreak leaf extensive damage ...
2,LSU QB beats out competitors to take home cove...,0,lsu qb beat competitor take home covet heisman...
3,Princess Charlotte takes center stage in Princ...,0,princess charlotte take center stage prince pr...
4,Trump moves into clear lead as Biden's approva...,0,trump move clear lead bidens approval rating p...
...,...,...,...
280,“It put me into a lot of danger and I hurt a l...,1,put lot danger hurt lot people along way jada ...
281,“Kris getting Tristan a deal with ESPN??? Afte...,1,kris get tristan deal espn twice betrayed daug...
282,“Beyoncé and Taylor just told y’all to shut up...,1,beyoncé taylor told you all shut behave one tw...
283,Court documents revealed that Joe and Sophie w...,1,court document reveal joe sophie go day mediat...


In [21]:
# tdf.to_csv('test_data_preprocessed.csv', index=False)

In [ ]:
#LSTM

In [24]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D
import tensorflow as tf

In [27]:
# #original data
# # train_data = df.drop("text", axis = 1)
# # test_data = tdf.drop("text", axis = 1)
# train_data.rename(columns={'text': 'headline'}, inplace=True)
# test_data.rename(columns={'text': 'headline'}, inplace=True)
df

,text,contains_gpe,sentiment,contains_numbers,contains_personal_pronouns,label,all_cleaned
0,Should I Get Bings,0,0.0000,0,0,1,get bings
1,Which TV Female Friend Group Do You Belong In,0,0.4939,0,1,1,tv female friend group belong
2,"The New ""Star Wars: The Force Awakens"" Trailer...",0,-0.5574,0,1,1,new star war force awakens trailer give chill
3,"This Vine Of New York On ""Celebrity Big Brothe...",0,0.6115,0,0,1,vine new york celebrity big brother fuck perfect
4,A Couple Did A Stunning Photo Shoot With Their...,0,-0.3400,0,0,1,couple stun photo shoot baby learn inoperable ...
...,...,...,...,...,...,...,...
31995,"To Make Female Hearts Flutter in Iraq, Throw a...",1,0.0000,0,0,0,make female heart flutter iraq throw shoe
31996,"British Liberal Democrat Patsy Calton, 56, die...",0,-0.6597,1,0,0,british liberal democrat patsy calton dy cancer
31997,Drone smartphone app to help heart attack vict...,0,-0.4019,0,0,0,drone smartphone app help heart attack victim ...
31998,"Netanyahu Urges Pope Benedict, in Israel, to D...",1,-0.3400,0,0,0,netanyahu urge pope benedict israel denounce iran


In [28]:
#cleaned datad
train_data = df.drop("text", axis = 1)
test_data = tdf.drop("headline", axis = 1)
train_data.rename(columns={'all_cleaned': 'headline'}, inplace=True)
test_data.rename(columns={'all_cleaned': 'headline'}, inplace=True)

In [29]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense

#cleaned datad
train_data = df.drop("text", axis = 1)
test_data = tdf.drop("headline", axis = 1)
train_data.rename(columns={'all_cleaned': 'headline'}, inplace=True)
test_data.rename(columns={'all_cleaned': 'headline'}, inplace=True)

full_data = train_data
corpus = full_data['headline'].tolist()
labels = full_data['label'].tolist()

# Use TfidfVectorizer to convert text to TF-IDF representations
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(corpus).toarray()  # Convert to dense array

# Convert labels to NumPy array
labels = np.array(labels)

# Build a simple neural network
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_tfidf.shape[1]))  # Input layer
model.add(Dense(units=1, activation='sigmoid'))  # Output layer (binary classification)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the entire dataset
model.fit(X_tfidf, labels, epochs=5, batch_size=32)

# Evaluate the model on the test set
X_test_tfidf = tfidf_vectorizer.transform(test_data['headline'].tolist()).toarray()
labels_test = np.array(test_data['label'].tolist())

loss, accuracy = model.evaluate(X_test_tfidf, labels_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')


Epoch 1/5
1000/1000 [==============================] - 7s 4ms/step - loss: 0.2198 - accuracy: 0.9383
Epoch 2/5
1000/1000 [==============================] - 4s 4ms/step - loss: 0.0638 - accuracy: 0.9784
Epoch 3/5
1000/1000 [==============================] - 4s 4ms/step - loss: 0.0340 - accuracy: 0.9898
Epoch 4/5
1000/1000 [==============================] - 4s 4ms/step - loss: 0.0199 - accuracy: 0.9950
Epoch 5/5
9/9 [==============================] - 0s 4ms/step - loss: 0.6963 - accuracy: 0.8386
Test Loss: 0.6963, Test Accuracy: 0.8386


In [ ]:
#Model building

In [30]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

#cleaned datad
train_data = df.drop("text", axis = 1)
test_data = tdf.drop("headline", axis = 1)
train_data.rename(columns={'all_cleaned': 'headline'}, inplace=True)
test_data.rename(columns={'all_cleaned': 'headline'}, inplace=True)


full_data = train_data
corpus = full_data['headline'].tolist()
labels = full_data['label'].tolist()

# Use TfidfVectorizer to convert text to TF-IDF representations
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(corpus)

# Build a logistic regression model
logreg_model = LogisticRegression()

# Train the model on the entire dataset
logreg_model.fit(X_tfidf, labels)

# Separate testing data
test_corpus = test_data['headline'].tolist()
test_labels = test_data['label'].tolist()

# Use TfidfVectorizer to transform the testing data
X_test_tfidf = tfidf_vectorizer.transform(test_corpus)

# Make predictions on the testing set
y_test_pred = logreg_model.predict(X_test_tfidf)

# Evaluate the model on the testing set
test_accuracy = accuracy_score(test_labels, y_test_pred)
print(f'Testing Accuracy: {test_accuracy:.4f}')

# Display additional evaluation metrics
print('\nClassification Report:')
print(classification_report(test_labels, y_test_pred))


Testing Accuracy: 0.8281

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.68      0.71        87
           1       0.86      0.89      0.88       198

    accuracy                           0.83       285
   macro avg       0.80      0.79      0.79       285
weighted avg       0.82      0.83      0.83       285



In [31]:
from sklearn.naive_bayes import MultinomialNB
# Build a Multinomial Naive Bayes model
nb_model = MultinomialNB()

# Train the model on the entire dataset
nb_model.fit(X_tfidf, labels)

# Separate testing data
test_corpus = test_data['headline'].tolist()
test_labels = test_data['label'].tolist()

# Use TfidfVectorizer to transform the testing data
X_test_tfidf = tfidf_vectorizer.transform(test_corpus)

# Make predictions on the testing set
y_test_pred = nb_model.predict(X_test_tfidf)

# Evaluate the model on the testing set
test_accuracy = accuracy_score(test_labels, y_test_pred)
print(f'Testing Accuracy: {test_accuracy:.4f}')

# Display additional evaluation metrics
print('\nClassification Report:')
print(classification_report(test_labels, y_test_pred))

Testing Accuracy: 0.8456

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.67      0.72        87
           1       0.86      0.92      0.89       198

    accuracy                           0.85       285
   macro avg       0.83      0.80      0.81       285
weighted avg       0.84      0.85      0.84       285



In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report

# --- Random Forest ---
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_tfidf, labels)
rf_predictions = rf_model.predict(X_test_tfidf)

# Evaluate Random Forest
rf_accuracy = accuracy_score(y_test_pred, rf_predictions)
print("Random Forest Testing Accuracy:", rf_accuracy)
print("\nRandom Forest Classification Report:")
print(classification_report(test_labels, rf_predictions))

Random Forest Testing Accuracy: 0.8701754385964913

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.57      0.60        87
           1       0.82      0.85      0.83       198

    accuracy                           0.76       285
   macro avg       0.72      0.71      0.72       285
weighted avg       0.76      0.76      0.76       285



In [38]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

#cleaned datad
train_data = df.drop("text", axis = 1)
test_data = tdf.drop("headline", axis = 1)
train_data.rename(columns={'all_cleaned': 'headline'}, inplace=True)
test_data.rename(columns={'all_cleaned': 'headline'}, inplace=True)
y = df['label']


full_data = train_data
corpus = full_data['headline'].tolist()
labels = full_data['label'].tolist()

# Use TfidfVectorizer to convert text to TF-IDF representations
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(corpus)

# --- Random Forest ---
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_tfidf, labels)

# Feature Importance
feature_names = tfidf_vectorizer.get_feature_names_out()
feature_importance = rf_model.feature_importances_

# Create a DataFrame to display feature importance
df_feature_importance = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
df_feature_importance = df_feature_importance.sort_values(by='Importance', ascending=False)

# Display the top 10 features
print("Top 10 Important Features (Random Forest):")
print(df_feature_importance.head(10))

# Make predictions on the entire dataset
y_predictions = rf_model.predict(X_tfidf)

# Evaluate Random Forest
rf_accuracy = accuracy_score(y, y_predictions)
print("\nRandom Forest Accuracy:", rf_accuracy)
print("\nRandom Forest Classification Report:")
print(classification_report(y, y_predictions))

Top 10 Important Features (Random Forest):
         Feature  Importance
17115      thing    0.022451
1491          be    0.016849
12693     people    0.014395
147     actually    0.012254
9404        know    0.011790
10289       make    0.011291
17231       time    0.011052
9322        kill    0.008786
2915   character    0.008476
9883        like    0.008457

Random Forest Accuracy: 1.0

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16001
           1       1.00      1.00      1.00     15999

    accuracy                           1.00     32000
   macro avg       1.00      1.00      1.00     32000
weighted avg       1.00      1.00      1.00     32000



In [34]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score

# Assuming 'df' is your DataFrame with 'headline' and 'label' columns
# You may need to preprocess the text before vectorization

# Cleaned data
train_data = df.drop("text", axis=1)
test_data = tdf.drop("headline", axis=1)
train_data.rename(columns={'all_cleaned': 'headline'}, inplace=True)
test_data.rename(columns={'all_cleaned': 'headline'}, inplace=True)

# Combine training and testing data
full_data = pd.concat([train_data, test_data], ignore_index=True)

# Use TfidfVectorizer to convert text to TF-IDF representations
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(full_data['headline'])

# --- Random Forest ---
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation
# cv_accuracy = cross_val_score(rf_model, X_tfidf[:len(train_data)], train_data['label'], cv=5, scoring='accuracy')

# Make predictions on the entire dataset
y_predictions = rf_model.fit(X_tfidf[:len(train_data)], train_data['label']).predict(X_tfidf)

# Evaluate Random Forest
rf_accuracy = accuracy_score(full_data['label'], y_predictions)
# print("\nRandom Forest Accuracy (Cross-Validation):", cv_accuracy.mean())
print("\nRandom Forest Classification Report:")
print(classification_report(full_data['label'], y_predictions))

# Confusion matrix
conf_matrix = confusion_matrix(full_data['label'], y_predictions)
print('\nConfusion Matrix:')
print(conf_matrix)

# Misclassified examples
misclassified_indices = (full_data['label'] != y_predictions)
misclassified_examples = full_data.loc[misclassified_indices, ['headline', 'label']]

# Display misclassified examples
print('\nMisclassified Examples:')
print(misclassified_examples)


Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16088
           1       1.00      1.00      1.00     16197

    accuracy                           1.00     32285
   macro avg       1.00      1.00      1.00     32285
weighted avg       1.00      1.00      1.00     32285


Confusion Matrix:
[[16052    36]
 [   34 16163]]

Misclassified Examples:
                                                headline  label
32005  father deathbed confession rattle family decad...      0
32008  big mistake parent make christmastime accord p...      0
32009  expert weighs debate go viral dad keep soninla...      0
32015  ryan oneals love story costar ali macgraw pay ...      0
32020  navy dad surprise family tear stadium heartstr...      0
...                                                  ...    ...
32263  julia claimed rapper want speak often buy pair...      1
32266  scott previously confess find ageappropri